In [1]:
# OpenCV-Python
import cv2
import numpy as np
import math
import sys, time
# Read image
img = cv2.imread('input_image.jpeg')
# Scale factor
ratio = 2
# Coefficient
a = -1/2

In [2]:
# Interpolation kernel
# a = -0.5(Keys)
def u(s,a):
    if (abs(s) >=0) & (abs(s) <=1):
        return (a+2)*(abs(s)**3)-(a+3)*(abs(s)**2)+1
    elif (abs(s) > 1) & (abs(s) <= 2):
        return a*(abs(s)**3)-(5*a)*(abs(s)**2)+(8*a)*abs(s)-4*a
    return 0

In [3]:
# Paddig ( p = 2)
def padding(img,H,W,C):
    zimg = np.zeros((H+4,W+4,C))
    zimg[2:H+2,2:W+2,:C] = img
    #Pad the first/last two col and row
    zimg[2:H+2,0:2,:C]=img[:,0:1,:C]          # the leftmost two col
    zimg[H+2:H+4,2:W+2,:]=img[H-1:H,:,:]      # the top two rows
    zimg[2:H+2,W+2:W+4,:]=img[:,W-1:W,:]      # the rightmost two col     
    zimg[0:2,2:W+2,:C]=img[0:1,:,:C]          # the bottom two rows          
    #Pad the missing eight points                        
    zimg[0:2,0:2,:C]=img[0,0,:C]              # down left corner (4 points)               
    zimg[H+2:H+4,0:2,:C]=img[H-1,0,:C]        # up left corner   (4 points)                    
    zimg[H+2:H+4,W+2:W+4,:C]=img[H-1,W-1,:C]  # up right corner  (4 points)                
    zimg[0:2,W+2:W+4,:C]=img[0,W-1,:C]        # down right corner(4 points)  
    return zimg

In [4]:
# https://github.com/yunabe/codelab/blob/master/misc/terminal_progressbar/progress.py
# def get_progressbar_str(progress):
#    END = 170
#    MAX_LEN = 30
#    BAR_LEN = int(MAX_LEN * progress)
#    return ('Progress:[' + '=' * BAR_LEN +
#            ('>' if BAR_LEN < MAX_LEN else '') +
#            ' ' * (MAX_LEN - BAR_LEN) +
#            '] %.1f%%' % (progress * 100.))

In [5]:
# Bicubic operation
def bicubic(img, ratio, a):
    #tuple with array dimensions
    H,W,C = img.shape

    img = padding(img,H,W,C)
    #Create new image
    dH = int(H*ratio) #rounds down to the nearest integer
    dW = int(W*ratio)
    dst = np.zeros((dH, dW, 3))
    h = 1/ratio
    #print('Start bicubic interpolation')
    #inc = 0 #for the progressbar
    for c in range(C):
        for j in range(dH):
            for i in range(dW):
                x, y = i * h + 2 , j * h + 2
                x1 = 1 + x - math.floor(x)
                x2 = x - math.floor(x)
                x3 = math.floor(x) + 1 - x
                x4 = math.floor(x) + 2 - x
                y1 = 1 + y - math.floor(y)
                y2 = y - math.floor(y)
                y3 = math.floor(y) + 1 - y
                y4 = math.floor(y) + 2 - y
                mat_l = np.array([[u(x1,a),u(x2,a),u(x3,a),u(x4,a)]])
                # matrix with the nearest 16 points of the new pixel
                mat_m = np.array([[img[int(y-y1),int(x-x1),c],img[int(y-y2),int(x-x1),c],img[int(y+y3),int(x-x1),c],img[int(y+y4),int(x-x1),c]],
                                   [img[int(y-y1),int(x-x2),c],img[int(y-y2),int(x-x2),c],img[int(y+y3),int(x-x2),c],img[int(y+y4),int(x-x2),c]],
                                   [img[int(y-y1),int(x+x3),c],img[int(y-y2),int(x+x3),c],img[int(y+y3),int(x+x3),c],img[int(y+y4),int(x+x3),c]],
                                   [img[int(y-y1),int(x+x4),c],img[int(y-y2),int(x+x4),c],img[int(y+y3),int(x+x4),c],img[int(y+y4),int(x+x4),c]]])
                mat_r = np.array([[u(y1,a)],[u(y2,a)],[u(y3,a)],[u(y4,a)]])
                dst[j, i, c] = np.dot(np.dot(mat_l, mat_m),mat_r)
                # Print progress
                # inc = inc + 1
                # sys.stderr.write('\r\033[K' + get_progressbar_str(inc/(C*dH*dW)))
                # sys.stderr.flush()
    sys.stderr.write('\n')
    sys.stderr.flush()
    return dst

In [6]:
import time
start_time = time.time()
dst = bicubic(img, ratio, a)
stop_time = time.time()
sw_exec_time = stop_time - start_time
print('Bicubic CPU Python execution time: ',sw_exec_time)
#cv2.imshow('output_image',dst)
#cv2.waitKey(0)
#print('Completed!')
cv2.imwrite('output_image.jpeg', dst)

Bicubic CPU Python execution time:  29.069288730621338


True